In [68]:
import pandas as pd
import numpy as np

from sklearn.feature_extraction.text import TfidfVectorizer 
from sklearn.metrics.pairwise import cosine_similarity

import nltk
import re
import gensim
import pickle
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
nltk.download('punkt')
from gensim.models.doc2vec import Doc2Vec, TaggedDocument

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/mariiabogdanova/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [69]:
CLEANFILE = "./Data/cleanLPW.txt"

documents = []
with open(CLEANFILE, 'r', encoding="utf8", errors="ignore") as inputfile:
    lines = inputfile.readlines()
    for line in lines[1:-1]:
        documents.append(line)

# Document Similarity: word2vec

In [70]:
documents_df = pd.DataFrame(documents,columns=['documents'])

# removing special characters and stop words from the text

stopwords_list = stopwords.words('english')
documents_df['documents_cleaned'] = documents_df.documents.apply(lambda x: " ".join(re.sub(r'[^a-zA-Z]',' ',w).lower() for w in x.split() if re.sub(r'[^a-zA-Z]',' ',w).lower() not in stopwords_list) )
documents_df.head(5)

,documents,documents_cleaned
0,The national executive of the strife-torn Demo...,national executive strife torn democrats last ...
1,Cash-strapped financial services group AMP has...,cash strapped financial services group amp she...
2,The United States government has said it wants...,united states government said wants see presid...
3,A radical armed Islamist group with ties to Te...,radical armed islamist group ties tehran baghd...
4,Washington has sharply rebuked Russia over bom...,washington sharply rebuked russia bombings geo...


In [71]:
tfidfvectoriser = TfidfVectorizer(max_features = 64)
tfidfvectoriser.fit(documents_df.documents_cleaned)
tfidf_vectors = tfidfvectoriser.transform(documents_df.documents_cleaned)
tfidf_vectors = tfidf_vectors.toarray()

In [72]:
# tokenize and pad every document to make them of the same size
tokenizer = Tokenizer()
tokenizer.fit_on_texts(documents_df.documents_cleaned)
tokenized_documents = tokenizer.texts_to_sequences(documents_df.documents_cleaned)
tokenized_paded_documents = pad_sequences(tokenized_documents,maxlen = 64,padding = 'post')
vocab_size = len(tokenizer.word_index)+1

print(tokenized_paded_documents[0])

[190  60 424 425  37   4  19 191 192 426 193  27  14 194  93  94  20 427
  95 428 429 430  10   1  96 431  95 432 195  10   1  61  96 190  60   4
  19 196 197  97   1 198 199  94  20 200  14  93 433 434  20  98  99 100
 435 201 202 436   0   0   0   0   0   0]


In [73]:
W2V_PATH = "Binaries/GoogleNews-vectors-negative300.bin.gz"
model_w2v = gensim.models.KeyedVectors.load_word2vec_format(W2V_PATH, binary=True)

In [74]:
embedding_matrix = np.zeros((vocab_size,300))
for word,i in tokenizer.word_index.items():
    if word in model_w2v:
        embedding_matrix[i] = model_w2v[word]
        
# creating document-word embeddings
document_word_embeddings = np.zeros((len(tokenized_paded_documents),64,300))

for i in range(len(tokenized_paded_documents)):
    for j in range(len(tokenized_paded_documents[0])):
        document_word_embeddings[i][j] = embedding_matrix[tokenized_paded_documents[i][j]]
        
# tf-idf vectors do not keep the original sequence of words, converting them into actual word sequences from the documents
document_embeddings = np.zeros((len(tokenized_paded_documents),300))
words = tfidfvectoriser.get_feature_names()

for i in range(len(document_word_embeddings)):
    for j in range(len(words)):
        document_embeddings[i] += embedding_matrix[tokenizer.word_index[words[j]]]*tfidf_vectors[i][j]
        
document_embeddings_word2vec = document_embeddings/np.sum(tfidf_vectors,axis=1).reshape(-1,1)
    

/Users/mariiabogdanova/miniconda3/lib/python3.8/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [78]:
pairwise_similarities_word2vec = cosine_similarity(document_embeddings_word2vec)

# with open('Binaries/pairwise_similarities_word2vec.pickle','wb') as f:
#     pickle.dump(pairwise_similarities_word2vec, f)

# Document Similarity: doc2vec

In [79]:
tagged_data = [TaggedDocument(words = word_tokenize(doc), tags = [i]) for i, doc in enumerate(documents_df.documents_cleaned)]

model_d2v = Doc2Vec(vector_size = 100, alpha = 0.025, min_count=1)
model_d2v.build_vocab(tagged_data)

for epoch in range(100):
    model_d2v.train(tagged_data, total_examples = model_d2v.corpus_count, epochs = model_d2v.epochs)

document_embeddings_doc2vec = np.zeros((documents_df.shape[0],100))
for i in range(len(document_embeddings)):
    document_embeddings_doc2vec[i] = model_d2v.docvecs[i]

<ipython-input-79-2999ae2f9c8b>:11: DeprecationWarning: Call to deprecated `docvecs` (The `docvecs` property has been renamed `dv`.).
  document_embeddings_doc2vec[i] = model_d2v.docvecs[i]


In [80]:
pairwise_similarities_doc2vec = cosine_similarity(document_embeddings_doc2vec)

# with open('Binaries/pairwise_similarities_doc2vec.pickle','wb') as f:
#     pickle.dump(pairwise_similarities_doc2vec, f)